# Goodman's Maximum Radial Velocity Precision

Let's start by defining some simple functions:

In [26]:
from astropy import constants
from astropy import units as u
import numpy as np
import sys
sys.path.append('/user/simon/development/soar/goodman')
from pipeline.core import SpectroscopicMode
# print(constants.c)
kms = u.km / u.s

gratings = [400, 600, 930, 1200, 1800, 2100, 2400] 
grating_modes = {400:['m1', 'm2'],
                 600:['UV', 'Blue', 'Mid', 'Red'],
                 930:['m1', 'm2', 'm3', 'm4', 'm5', 'm6'],
                 1200:['m0','m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7'],
                 1800:['Custom'],
                 2100:['Custom'],
                 2400:['Custom']}
grating_dispersion = {400:1.0,
                      600:0.65,
                      930:0.42,
                      1200:0.31,
                      1800:0.19,
                      2100:0.15,
                      2400:0.12}

slit_sizes = [0.45, 1.0, 3.0, 5.0] * u.arcsec

pixel_size = 15 * u.micrometer
pixel_scale = 0.15 * u.arcsec
goodman_focal_length = 377.3 * u.mm

def get_central_wavelength(grating, grt_ang, cam_ang):
    """Calculates the central wavelength for a given spectroscopic mode

    The equation used to calculate the central wavelength is the following


    .. math::
        \\lambda_{central} = \\frac{1e6}{GRAT}
        \\sin\\left(\\frac{\\alpha \\pi}{180}\\right) +
        \\sin\\left(\\frac{\\beta \\pi}{180}\\right)


    Args:
        grating (str): Grating frequency as a string. Example '400'.
        grt_ang (str): Grating Angle as a string. Example '12.0'.
        cam_ang (str): Camera Angle as a string. Example '20.0'

    Returns:
        central_wavelength (float): Central wavelength as a float value.

    """

    grating_frequency = float(grating)
    alpha = float(grt_ang)
    beta = float(cam_ang) - float(grt_ang)

    central_wavelength = (1e6 / grating_frequency) * \
                         (np.sin(alpha * np.pi / 180.) +
                          np.sin(beta * np.pi / 180.))

    # print('Found {:.3f} as central wavelength'.format(central_wavelength))

    return central_wavelength

def rv_value(rest, measured):
    """Calculates Radial Velocity based using rest frame wavelength value versus measured"""
    print("Shift for {:s}: {:s}".format(str(rest.value), str(rest.value - measured)))
    rv = ((measured - rest.value)/rest.value) * constants.c.value
    # print(rv * constants.c.unit)
    rm_ms = rv * constants.c.unit
    return rm_ms.to(kms)

In [31]:
def print_table_(gratings, modes, cws, disps, rs, speeds):
    field_length = []
    for field in [gratings, modes, cws, disps, rs, speeds]:
        lengths = [len(i) for i in field]
        field_length.append(np.max(lengths))
    line_sep = "+"
    for fl in field_length:
        line_sep += '-' * fl + '+'
    print(line_sep)


In [32]:
spec_mode = SpectroscopicMode()
# print("+---------+------+--------------------+------------+---------------------+-------+")
# print("| Grating | Mode | Central Wavelength | Dispersion | Resolving Power (R) | Speed |")
# print("+=========+======+====================+============+=====================+=======+")
center_wav = []
resolving_power = []
speeds = []
for grat in gratings:
    for wavmode in grating_modes[grat]:
#         print(repr(grat), repr(wavmode))
        try:
            cam_targ, grat_targ = spec_mode.get_cam_grt_targ_angle(grating=grat, mode=wavmode)
            cw = get_central_wavelength(grating=grat, grt_ang=grat_targ, cam_ang=cam_targ)
            r = cw / (grating_dispersion[grat] / 3.)
            v = constants.c / r
#             print("\tGrating: {:d} \n\tMode: {:s}\n\tCentral Wavelength: {:.3f}\n\tDispersion: {:.3f} \n\tR: "
#                   "{:.3f}\n\tSpeed: {:.3f}".format(grat,
#                                                    wavmode,
#                                                    cw,
#                                                    grating_dispersion[grat],
#                                                    r,
#                                                    v.to('km/s')))
            
#             print("| {:d} | {:s} | {:.3f} | {:.3f} | {:.3f} |  {:.3f} |".format(grat,
#                                                    wavmode,
#                                                    cw,
#                                                    grating_dispersion[grat],
#                                                    r,
#                                                    v.to('km/s')))
            center_wav.append(cw)
            resolving_power.append(r)
            speeds.append(v)
#             print("+---------+------+--------------------+------------+---------------------+-------+")
        except TypeError:
            print("Error")
print_table_(gratings, grating_modes, center_wav, grating_dispersion, resolving_power, speeds)
          

Error
Error
Error


TypeError: object of type 'int' has no len()